# <span style='color: tomato'>SECTION 7</span>

## Optical FLow

https://en.wikipedia.org/wiki/Optical_flow

### Lucas-Kanade function

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Parameters for ShiTomasi corner detection (good features to track paper)
corner_track_params = dict(maxCorners = 10,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

In [3]:
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (200,200),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10,0.03))

In [7]:
# Capture the video
cap = cv2.VideoCapture(1)

# Grab the very first frame of the stream
if cap.isOpened():
   ret, prev_frame = cap.read()
   if ret:
    # Grab a grayscale image (We will refer to this as the previous frame)
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    # Grabbing the corners
    prevPts = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_track_params)

    # Create a matching mask of the previous frame for drawing on later
    mask = np.zeros_like(prev_frame)
    print("capture frame")
   else:
      print("Error : Failed to capture frame")
else:
   print("Cannot open camera")



while True:
    
    # Grab current frame
    ret,frame = cap.read()
    
    # Grab gray scale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate the Optical Flow on the Gray Scale Frame
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)
    
    # Using the returned status array (the status output)
    # status output status vector (of unsigned chars); each element of the vector is set to 1 if
    # the flow for the corresponding features has been found, otherwise, it is set to 0.
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]
    
    # Use ravel to get points to draw lines and circles
    for i,(new,prev) in enumerate(zip(good_new,good_prev)):
        
        x_new,y_new = new.ravel()
        x_prev,y_prev = prev.ravel()
        
        # ERROR FIX
        x_new = int(x_new)
        y_new = int(y_new)
        x_prev = int(x_prev)
        y_prev = int(y_prev)
        
        # Lines will be drawn using the mask created from the first frame
        mask = cv2.line(mask, (x_new,y_new),(x_prev,y_prev), (0,255,0), 3)
        
        # Draw red circles at corner points
        frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)
    
    # Display the image along with the mask we drew the line on.
    img = cv2.add(frame,mask)
    cv2.imshow('frame',img)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
   
    # Now update the previous frame and previous points
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)
    
    
cv2.destroyAllWindows()
cap.release()

capture frame


In [6]:
cap = cv2.VideoCapture(0)

ret, frame1 = cap.read()

prvsImg = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255


while True:
  ret, frame2 = cap.read()
  
  nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
  
  flow = cv2.calcOpticalFlowFarneback(prvsImg,nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)
  
  mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1],angleInDegrees=True)
  
  hsv_mask[:,:,0] = ang/2
  hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX) # <--- ???
  
  bgr = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)
  cv2.imshow('frame',bgr)
  
  k = cv2.waitKey(10) & 0xFF
  if k == 27:
    break
  
  
  prvsImg = nextImg
  
cap.release()
cv2.destroyAllWindows()

## IMAGE TRACKING

### MEANSHIFT ALGORITHM

In [7]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
cap = cv2.VideoCapture(1)

ret, frame = cap.read()

face_cascade = cv2.CascadeClassifier('DATA/haarcascades/haarcascade_frontalface_default.xml')
face_rects = face_cascade.detectMultiScale(frame)

(face_x, face_y, w, h) = tuple(face_rects[0])
track_window = (face_x, face_y, w, h)


roi = frame[face_y:face_y+h, face_x:face_x+w]

hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)

roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

cv2.normalize(roi_hist, roi_hist, 0,255,cv2.NORM_MINMAX)

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

while True:
  ret, frame = cap.read()
  
  if ret:
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    dst = cv2.calcBackProject([hsv],[0], roi_hist, [0,180], 1)
    
    
    #########################
    
    # ret, track_window = cv2.meanShift(dst, track_window, term_crit)
    
    # x,y,w,h = track_window
    # img2 = cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 3)
    
    ret, track_window = cv2.CamShift(dst, track_window, term_crit)
    
    pts = cv2.boxPoints(ret)
    pts = np.int0(pts)
    img2 = cv2.polylines(frame, [pts], True, (0,0,255),3)
    
    #########################
    cv2.imshow('img', img2)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
      break
    
  else:
    break
cv2.destroyAllWindows()
cap.release()





C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12436\1271545858.py:41: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  pts = np.int0(pts)


## API TRACKERS

### MIL TRACKER http://faculty.ucmerced.edu/mhyang/papers/cvpr09a.pdf

### KCF TRACKER https://arxiv.org/abs/1404.7584

### TDL TRACKER https://ieeexplore.ieee.org/document/6104061

### MEDIAN FLOW TRACKER https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.231.4285&rep=rep1&type=pdf

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
def ask_for_tracker():
    print("Welcome! What Tracker API would you like to use?")
    print("Enter 0 for BOOSTING: ")
    print("Enter 1 for MIL: ")
    print("Enter 2 for KCF: ")
    print("Enter 3 for TLD: ")
    print("Enter 4 for MEDIANFLOW: ")
    choice = input("Please select your tracker: ")
    
    if choice == '0':
        tracker = cv2.legacy.TrackerBoosting_create()
    if choice == '1':
        tracker = cv2.TrackerMIL_create()
    if choice == '2':
        tracker = cv2.legacy.TrackerKCF_create()
    if choice == '3':
        tracker = cv2.legacy.TrackerTLD_create()
    if choice == '4':
        tracker = cv2.TrackerMedianFlow_create()


    return tracker

In [3]:
tracker = ask_for_tracker()
tracker

Welcome! What Tracker API would you like to use?
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 


UnboundLocalError: cannot access local variable 'tracker' where it is not associated with a value

In [6]:
str(tracker).split()[1][4:]

'TrackerMIL'

In [14]:
import time

tracker = ask_for_tracker()
tracker_name = str(tracker).split()[1][4:]

# Read video
cap = cv2.VideoCapture(0)

# Wczytanie pierwszej klatki przed pętlą
ret, frame = cap.read()


# Special function allows us to draw on the very first frame our desired ROI
if ret:
    roi = cv2.selectROI(frame, False)

    # Initialize tracker with first frame and bounding box
    ret = tracker.init(frame, roi)

while True:
    # Read a new frame
    ret, frame = cap.read()
    
    
    # Update tracker
    success, roi = tracker.update(frame)
    
    # roi variable is a tuple of 4 floats
    # We need each value and we need them as integers
    (x,y,w,h) = tuple(map(int,roi))
    
    # Draw Rectangle as Tracker moves
    if success:
        # Tracking success
        p1 = (x, y)
        p2 = (x+w, y+h)
        cv2.rectangle(frame, p1, p2, (0,255,0), 3)
    else :
        # Tracking failure
        cv2.putText(frame, "Failure to Detect Tracking!!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)

    # Display tracker type on frame
    cv2.putText(frame, tracker_name, (20,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),3);

    # Display result
    cv2.imshow(tracker_name, frame)

    # Exit if ESC pressed
    k = cv2.waitKey(1) & 0xff
    if k == 27 : 
        break
        
cap.release()
cv2.destroyAllWindows()

Welcome! What Tracker API would you like to use?
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 
